In [ ]:
from daphne import daphne
import os

In [ ]:
directory = 'programs/tests/hoppl-deterministic'
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/{}/'.format(directory))

In [ ]:
i=3
cwd='/Users/gw/repos/daphne'
exp = daphne(['desugar-hoppl', '-i', '../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne'.format(i)])
exp



In [ ]:
desugared_ast_json_fname = '/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne.json'.format(i)
with open(desugared_ast_json_fname, 'w') as f:
    json.dump(exp, f)

In [ ]:
%ls /Users/gw/repos/daphne/../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic

In [ ]:
class Env():
    "An environment: a dict of {'var': val} pairs, with an outer Env."
    def __init__(self, parms=(), args=(), outer=None):
        self.data = pmap(zip(parms, args))
        self.outer = outer
        if outer is None:
            self.level = 0
        else:
            self.level = outer.level+1
    def __getitem__(self, item):
        return self.data[item]
    def find(self, var):
        "Find the innermost Env where var appears."
        if (var in self.data):
            return self
        else:
            if self.outer is not None:
                return self.outer.find(var)
            else:
                raise RuntimeError('var "{}" not found in outermost scope'.format(var))
    
    def print_env(self, print_lowest=False):
        print_limit = 1 if print_lowest == False else 0
        outer = self
        while outer is not None:
            if outer.level >= print_limit:
                print('Scope on level ', outer.level)
                if 'f' in outer:
                    print('Found f, ')
                    print(outer['f'].body)
                    print(outer['f'].parms)
                    print(outer['f'].env)
                print(outer,'\n')
            outer = outer.outer

In [ ]:
import copy

class Procedure(object):
    "A user-defined Scheme procedure."
    def __init__(self, parms, body, env):
        self.parms, self.body, self.env = parms, body, env
    def __call__(self, *args): 
        new_env = copy.deepcopy(self.env)
        return eval(self.body, Env(self.parms, args, new_env)) # [0]

In [ ]:
from pyrsistent import pmap

import torch
import torch.distributions as dist

In [ ]:
def push_addr(alpha, value):
    return alpha + value

penv = {
    '+': torch.add,
    '*':torch.mul,
#     'push-address':push_addr
}

In [ ]:
def standard_env():
    "An environment with some Scheme standard procedures."
    env = Env(penv.keys(), penv.values())
    return env

env = standard_env()
env.find('+')
#['+'](torch.tensor(0.),torch.tensor(1.))

In [ ]:
def eval(x,env=standard_env(),sigma=None):

    print('x',x)
    if isinstance(x,str):
        return env.find(x)[x], sigma
    elif not isinstance(x,list):
        return torch.tensor(x),sigma
    
    op, param, *args = x
    
    if False:
        pass
    
    elif op == 'push-address':
        return '', sigma
    elif op == 'fn':
        print('args',args)
#         param, body = args
        body = args[0]
        return Procedure(param, body, env), sigma # has eval in it
        # param ['alpha', 'x']
        # body ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']
        # env
    
    else:
        print('in else. x',x)
        proc, _ = eval(op,env,sigma)
        vals = [''] # TODO: address for fn???
        vals.extend([eval(arg, env, sigma)[0] for arg in args])
        print('vals',vals)

        if isinstance(proc, Procedure): # lambdas, not primitives
            print('in Procedure',proc, vals)
            r, _ = proc(*vals)
        else:
            r = proc(*vals[1:]) # primitives
            print('in primitives',proc, vals)
            
        return r, sigma
    

# eval(['*','a0',3,3])  

# why vals only tensor(8.), where did push add go?

In [ ]:
exp

In [ ]:
proc, sigma = eval(exp)
proc('')

In [ ]:
exp_sq_8 = [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']],
   ['push-address', 'alpha', 'addr1'],
   9]

eval(exp_sq_8)#(torch.tensor(8.))

In [ ]:
exp_inner = ['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
eval(exp_inner)#(torch.tensor(8.))

# test driven development

In [ ]:
import sys
sys.setrecursionlimit(10000)

## cons

In [ ]:
exp = [tensor(0.), tensor([1,2,3])]

torch.cat((torch.tensor([exp[0]]), exp[1]), dim=0)

In [ ]:
def cons(*exp):
    try:
        return torch.cat((torch.tensor([exp[1]]), exp[0]), dim=0)
    except:
        return torch.cat((torch.tensor([exp[1]]), torch.tensor(exp[0])), dim = 0)

## hash-map

In [ ]:
# eval_hoppl(['hash-map','','a',1,'b',2])

In [ ]:
# from old_primitives import penv
# from torch import tensor

# vals = ['',tensor(1.),tensor(2.)]
# penv['+'](vals[1:])

In [ ]:
# fname = 'programs/tests/hoppl-deterministic/test_{}.daphne.json'.format(i)

# with open(fname) as f:
#      exp = json.load(f)
# exp
# # 

## empty

In [ ]:
from primitives import penv
penv['empty?']([tensor([1])])

In [ ]:
['', tensor([])][1:]

In [ ]:
eval_hoppl(['empty?','',['vector','']],do_log=True)

In [ ]:
args=[[tensor(0.), tensor(1.)]]

[eval_hoppl(arg)[0] for arg in args]

In [ ]:
eval_hoppl(['empty?','',[tensor(0.),tensor(1.)]],do_log=True)

In [ ]:
eval_hoppl(['vector','',tensor(1),tensor(2)])

## cons/conj
* 12 cons and map. cons primitive can't handle  
  * vals[1:] [tensor([]), tensor([4])] 
  * because can't do 

In [ ]:
tensor(tensor([]))

In [ ]:
item, vector = [tensor([]), tensor([4])]
penv['cons']([tensor([]), tensor([4])])

In [ ]:
item = tensor([1])
vector = tensor([4,5])
torch.cat((torch.tensor(item), vector), dim=0)

In [ ]:
torch.tensor(item)

## deterministic-hoppl tests

In [36]:
from tests import load_truth, is_tol
import os, json
from daphne import daphne
import torch
from torch import tensor

import importlib
import evaluator
importlib.reload(evaluator)
from evaluator import evaluate, run_deterministic_tests, ast_helper, eval_hoppl

import primitives
importlib.reload(primitives)

<module 'primitives' from '/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/primitives.py'>

In [56]:
i=6
fname='test_{}.daphne'.format(i)
exp = ast_helper(fname,directory='programs/tests/probabilistic/')
exp

['fn',
 ['alpha'],
 [['fn',
   ['alpha', 'get-sig'],
   ['sample',
    ['push-address', 'alpha', 'addr1'],
    ['normal',
     ['push-address', 'alpha', 'addr2'],
     0.0,
     ['get-sig', ['push-address', 'alpha', 'addr3'], 1.2]]]],
  ['push-address', 'alpha', 'addr0'],
  ['fn',
   ['alpha', 'var'],
   [['fn',
     ['alpha', 'var', 'get-sig'],
     ['*', ['push-address', 'alpha', 'addr5'], 'var', 'var']],
    ['push-address', 'alpha', 'addr4'],
    'var',
    ['fn',
     ['alpha', 'var', 'get-sig'],
     ['*', ['push-address', 'alpha', 'addr6'], 'var', 'var']]]]]]

In [16]:
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/')
truth = load_truth('programs/tests/hoppl-deterministic/test_{}.truth'.format(i))
truth

1

In [60]:
ret = evaluate(exp, do_log=False)
ret
# assert(is_tol(ret, truth))
# 4,6,8 quote a and b. daphne output good?

tensor(-0.5153)

## all tests

In [21]:
%cd /Users/gw/repos/prob_prog/hw/hw5/CS532-HW5
run_deterministic_tests()

/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5
FOPPL test 1 passed
FOPPL test 2 passed
FOPPL test 3 passed
FOPPL test 4 passed
FOPPL test 5 passed
FOPPL test 6 passed
FOPPL test 7 passed
FOPPL test 8 passed
FOPPL test 9 passed
FOPPL test 10 passed
FOPPL test 11 passed
FOPPL test 12 passed
FOPPL test 13 passed
HOPPL test 1 passed
HOPPL test 2 passed
HOPPL test 3 passed
HOPPL test 4 failed to evaluate
HOPPL test 5 passed
HOPPL test 6 failed to evaluate
HOPPL test 7 passed
HOPPL test 8 failed to evaluate
HOPPL test 9 passed
HOPPL test 10 passed
HOPPL test 11 passed
HOPPL test 12 passed
All deterministic tests passed


/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/primitives.py:206: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat((vector,torch.tensor(item)), dim=0)


In [61]:
%cd /Users/gw/repos/prob_prog/hw/hw5/CS532-HW5
import evaluator
importlib.reload(evaluator)
evaluator.run_probabilistic_tests()

/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5
('normal', 5, 1.4142136)
p value 0.6860054999915197
('beta', 2.0, 5.0)
p value 0.6741978639149588
('exponential', 0.0, 5.0)
p value 0.033207612079003246
('normal', 5.3, 3.2)
p value 0.06612420192766066
('normalmix', 0.1, -1, 0.3, 0.9, 1, 0.3)
p value 0.6267830939943639
('normal', 0, 1.44)
p value 0.6772922643984287
All probabilistic tests passed
